# Example: Impact Fund Screening
In this example we score potential land acquisitions in batch. The workflow shows how to combine land-cover metrics, B-Score calculation and occurrence density.

In [18]:
from pathlib import Path
import pandas as pd
from verdesat.services.bscore import compute_bscores
from verdesat.biodiv.gbif_validator import OccurrenceService, plot_score_vs_density

In [19]:
geojson_path = "examples/reforestation_plots.geojson"

In [22]:
# Load AOIs from GeoJSON
from verdesat.geo.aoi import AOI
projects_path = geojson_path
projects = AOI.from_geojson(projects_path, id_col='ID2')

In [ ]:
df_scores = pd.read_csv('examples/examples/bscore.csv')
df_scores['id'] = df_scores['id'].astype(int)
df_scores

In [ ]:
from verdesat.analytics.stats import compute_summary_stats
stats_df = compute_summary_stats('examples/reforestation_ts.csv', decomp_dir='examples/decomp', period=12).to_dataframe()
merged = stats_df.merge(df_scores, left_on='Site ID', right_on='id')
merged.plot.scatter(x='bscore', y="Sen's Slope (NDVI/yr)", figsize=(6,4))
merged

In [ ]:
import geopandas as gpd, folium
projects = gpd.read_file(geojson_path)
center = [projects.geometry.centroid.y.mean(), projects.geometry.centroid.x.mean()]
m = folium.Map(location=center, zoom_start=15)
folium.Choropleth(geo_data=projects, data=df_scores, columns=['id','bscore'], key_on='feature.properties.ID2', fill_color='YlGn').add_to(m)
m.save('examples/projects_map.html')
m

In [24]:
occ = OccurrenceService()
import geopandas as gpd
densities = []
for aoi in projects:
    aoi_gdf = gpd.GeoDataFrame({'geometry': [aoi.geometry]}, crs='EPSG:4326')
    gdf = occ.fetch_occurrences(aoi.geometry)
    area_km2 = aoi_gdf.to_crs(epsg=6933).area.iloc[0] / 1e6
    densities.append(occ.occurrence_density_km2(gdf, area_km2))
plot_score_vs_density(df_scores['bscore'].tolist(), densities, 'examples/score_vs_density.png')

2025-07-28 19:29:36 [INFO] verdesat.biodiv.gbif_validator – Fetching occurrences since 2000 for bbox (-92.53968503301898, 16.790645159322896, -92.5365480721544, 16.792274)
2025-07-28 19:29:36 [INFO] verdesat.biodiv.gbif_validator – Fetched 12 GBIF records
2025-07-28 19:29:36 [INFO] verdesat.biodiv.gbif_validator – EBIRD_TOKEN not set; skipping eBird fallback
2025-07-28 19:29:36 [INFO] verdesat.biodiv.gbif_validator – Querying iNaturalist within bbox (-92.53968503301898, 16.790645159322896, -92.5365480721544, 16.792274)
2025-07-28 19:29:36 [INFO] pyinaturalist – Request:
GET https://api.inaturalist.org/v1/observations?d1=2000-01-01T00%3A00%3A00%2B01%3A00&nelat=16.792274&nelng=-92.5365480721544&swlat=16.790645159322896&swlng=-92.53968503301898
User-Agent: python-requests/2.32.4 pyinaturalist/0.20.1
Accept-Encoding: gzip, deflate, zstd
Accept: application/json
Connection: keep-alive

2025-07-28 19:29:37 [INFO] verdesat.biodiv.gbif_validator – Fetched 2 iNaturalist records
2025-07-28 19:29

The scatter plot compares biodiversity score with citizen-science occurrence density for each polygon.

The scatter plot above relates biodiversity score with NDVI trend for each candidate parcel. Higher B-Score coupled with a stable or positive slope indicates a strong conservation opportunity.